In [3]:
import sys
import typing as tp
import os
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px

build_dir = Path(__vsc_ipynb_file__).parent.parent / "build"


sys.path.append(str(build_dir / "pacer" / "gps-source"))
sys.path.append(str(build_dir / "pacer" / "geometry"))

In [ ]:
from _pacer_gps_source_impl import (
    read_dat_file,
    DatVersion,
    GPSSource,
    GPMFSource,
    SequentialGPSSource,
)
from _pacer_geometry_impl import (
    CoordinateSystem,
    GPSSample,
    Vec3f,
)

In [6]:
def get_dat_df(filename: str) -> pd.DataFrame:
    lats, lons, ts, speeds = [], [], [], []

    read_dat_file(
        filename,
        lambda s, t: (
            lats.append(s.latitude),
            lons.append(s.longitude),
            ts.append(t),
            speeds.append(s.full_speed),
        ),
        DatVersion.WITH_TIMESTAMP,
    )

    return pd.DataFrame(
        {"latitude": lats, "longitude": lons, "timestamp": ts, "speed": speeds}
    )


def display_dat_file(filename: str) -> None:
    df = get_dat_df(filename)
    return px.scatter_map(
        df, lat="latitude", lon="longitude", color="speed", map_style="outdoors"
    ).update_layout(height=800)

In [ ]:
display_dat_file(dat_files[0])

NameError: name 'dat_files' is not defined

In [2]:
import pacer._pacer_geometry_impl as pacer_geometry

ImportError: dlopen(/Users/denys/dev/pacer/.pixi/envs/default/lib/python3.13/site-packages/pacer/_pacer_geometry_impl.cpython-313-darwin.so, 0x0002): Library not loaded: @rpath/libpacer_geometry.dylib
  Referenced from: <5714260D-77FA-33F1-B9F7-C8438A9E6F57> /Users/denys/dev/pacer/.pixi/envs/default/lib/python3.13/site-packages/pacer/_pacer_geometry_impl.cpython-313-darwin.so
  Reason: tried: '/Users/denys/dev/pacer/.pixi/envs/default/bin/../lib/libpacer_geometry.dylib' (no such file), '/Users/denys/dev/pacer/.pixi/envs/default/bin/../lib/libpacer_geometry.dylib' (no such file)

In [6]:
display_dat_file(dat_files[1])

In [7]:
display_dat_file(dat_files[2])

In [8]:
display_dat_file(dat_files[3])

In [35]:
s = GPMFSource("/Users/denys/Library/CloudStorage/GoogleDrive-dendi239@gmail.com/My Drive/photos/2025-may-daytona-mk/GH020252.MP4")
points = list(read_all_gps_data(s))

No payload


In [41]:
points[3]

(GPSSample(lat=52.040616, lon=-0.7841079, altitude=78.482, full_speed=12.617, ground_speed=12.56),
 0.10007994220990321,
 0.13343992294653761)

In [44]:
points_df = pd.DataFrame(
    {
        "latitude": [p.latitude for p, *_ in points],
        "longitude": [p.longitude for p, *_ in points],
        # "timestamp": [p.timestamp for p, *_ in points],
        "speed": [p.full_speed for p, *_ in points],
    }
)

In [45]:
points_df

,latitude,longitude,speed
0,52.040607,-0.784021,11.612
1,52.040612,-0.784049,11.993
2,52.040615,-0.784077,12.159
3,52.040616,-0.784108,12.617
4,52.040615,-0.784139,12.889
...,...,...,...
9565,52.039395,-0.782697,17.971
9566,52.039369,-0.782693,17.683
9567,52.039344,-0.782694,17.147
9568,52.039315,-0.782701,16.859


In [49]:
px.scatter_map(
    points_df,
    lat="latitude",
    lon="longitude",
    color="speed",
    map_style="outdoors",
).update_layout(height=800)

In [20]:
df = get_dat_df(dat_files[3])
df

,latitude,longitude,timestamp,speed
0,52.040888,-0.784933,554420.72,0.038
1,52.040888,-0.784933,554420.76,0.046
2,52.040888,-0.784933,554420.80,0.039
3,52.040888,-0.784933,554420.84,0.052
4,52.040888,-0.784933,554420.88,0.067
...,...,...,...,...
7997,52.040412,-0.784953,554753.44,0.265
7998,52.040412,-0.784953,554753.48,0.234
7999,52.040412,-0.784953,554753.52,0.276
8000,52.040412,-0.784953,554753.56,0.323


In [21]:
df["timestamp"] - df.iloc[0]["timestamp"]

0         0.00
1         0.04
2         0.08
3         0.12
4         0.16
         ...  
7997    332.72
7998    332.76
7999    332.80
8000    332.84
8001    332.84
Name: timestamp, Length: 8002, dtype: float64

In [11]:
1702 / 8002

0.21269682579355162

In [12]:
df = get_dat_df(dat_files[1])
(
    df
    # .loc[
    #     lambda d: (
    #         d[["latitude", "longitude", "speed"]]
    #         == d[["latitude", "longitude", "speed"]].shift(1)
    #     ).any(axis="columns")
    # ]
    .pipe(
        lambda d: px.scatter(
            x=d["timestamp"],
            y=d["timestamp"].diff(),
            log_y=True,
            labels={"x": "timestamp", "y": "diff"},
            title="Time difference between points",
        )
    )
)

In [13]:
px.histogram(np.log(df["timestamp"].diff()) / np.log(10), nbins=300, labels={"x": "log10(diff)"}, title="Log10 of time difference between points")

/Users/denys/dev/pacer/.pixi/envs/default/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning:

divide by zero encountered in log



In [14]:
def read_all_gps_data(s: GPSSource) -> tp.Iterable[tuple[GPSSample, float, float]]:
    """
    Read all GPS data from the given GPSSource and return it as a pandas DataFrame.
    """
    gps_data = []
    while not s.IsEnd():
        s.Samples(lambda s, i, n: gps_data.append(s))
        for si in gps_data:
            yield si, *s.CurrentTimeSpan()
        gps_data.clear()
        s.Next()

In [15]:
s = GPMFSource("/Users/denys/Documents/gokarting-ui/GH010219.MP4")

In [16]:
df = pd.DataFrame(
    {
        "latitude": si.latitude,
        "longitude": si.longitude,
        "altitude": si.altitude,
        "3d_speed": si.full_speed * 3.6,
        "2d_speed": si.ground_speed * 3.6,
        "time_in": t0,
        "time_out": t1,
    }
    for si, t0, t1 in read_all_gps_data(s)
)

df

,latitude,longitude,altitude,3d_speed,2d_speed,time_in,time_out
0,52.041231,-0.785139,58.399,0.0000,0.000,0.0,1.0
1,52.041231,-0.785139,58.399,0.0000,0.000,0.0,1.0
2,52.041231,-0.785139,58.399,0.0000,0.000,0.0,1.0
3,52.041231,-0.785139,58.399,0.0000,0.000,0.0,1.0
4,52.041231,-0.785139,58.399,0.0000,0.000,0.0,1.0
...,...,...,...,...,...,...,...
9651,51.375913,-0.361280,20.541,40.1976,39.888,531.0,532.0
9652,51.375908,-0.361276,20.585,40.1904,40.212,531.0,532.0
9653,51.375904,-0.361271,20.609,39.9636,40.212,531.0,532.0
9654,51.375899,-0.361266,20.637,39.5460,39.996,531.0,532.0


In [17]:
fig = px.scatter(
    df.loc[lambda d: d["3d_speed"] > 0.1],
    x="longitude",
    y="latitude",
    color="3d_speed",
    color_continuous_scale="Viridis",
    title="GPS Data (Colored by 3D Speed)",
)
fig

In [18]:
px.line(df["3d_speed"], title="3D Speed over Time")